In [1]:
import sys, os, html, re, json, glob, base64, urllib.request, cv2
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np
from blend_modes import normal
from slpp import slpp as lua

In [36]:

# Make sure working directory is project root
if os.getcwd().rsplit('\\',1)[1]=="model_training":
    os.chdir( os.getcwd().rsplit('\\',1)[0] )

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# Define functions used throughout the document
def read_png(path):
    image = cv2.imread(path,-1)
    return cv2.cvtColor(image,cv2.COLOR_BGR2BGRA) if len(image.shape) > 2 and image.shape[2] == 3 else image

def read_jpeg(path):
    BGR = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    BGRA = cv2.cvtColor(BGR,cv2.COLOR_BGR2BGRA)
    BGRA[:,:,3] = 255
    return BGRA

def layer( image_a, image_b, ratio=1.):
    return normal(image_a.astype(float), image_b.astype(float), ratio).astype(np.uint8)

def save_model(model, model_name):
    model.save                             (os.path.join("model_training","models",model_name,"model.h5"))
    model.save                             (os.path.join("model_training","models",model_name,"model"))
    model.save_weights                     (os.path.join("model_training","models",model_name,"model_weights_checkpoint","model_weights"))
    tfjs.converters.save_keras_model(model, os.path.join("public","models",model_name))
    
def load_model(model_name):
    return tf.keras.models.load_model(os.path.join("model_training","models",model_name,"model"));

def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [74]:
print("3. Image generation has begun")

if not os.path.isfile(os.path.join("public","lookup_data","loot.json")):
    exit("Game data file from step 1 not found. Exiting.")
with open(os.path.join("public","lookup_data","loot.json")) as json_file:
    lookup_loot = json.load(json_file)

def generate_champion_and_skin_training_images( champion_image_path ):
    
    champion_id = champion_image_path.rsplit(os.path.sep,1)[1].split('.')[0]
    champion_data = list(filter(lambda row: row[0] == champion_id, lookup_loot["champions"]+lookup_loot["skins"]))[0]
    price = champion_data[2]
    is_legacy = champion_data[3] == 1
    asset_folder = "champions" if champion_id.endswith("000") else "skins"  
    
    if len(glob.glob(os.path.join("model_training","training_data",asset_folder,champion_id,"*.png"))) == 336:
        return
    
    # 1.A loading and preparing assets
    champion = cv2.copyMakeBorder(read_jpeg(champion_image_path), 0, 5, 5, 0, cv2.BORDER_CONSTANT,value=(0,0,0,255))
    shard_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","shard.png")), (385,385))
    permanent_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","permanent.png")), (385,385))
    token_6_border = read_png(os.path.join("model_training","assets","border_images","champion_token_6.png"))[5:74, :]
    token_6_border = cv2.resize(token_6_border, (385,385))
    token_7_border = read_png(os.path.join("model_training","assets","border_images","champion_token_7.png"))[5:74, :]
    token_7_border = cv2.resize(token_7_border, (385,385))
    quantity_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","quantity_background.png")), (385,385))

    if is_legacy: legacy_icon = read_png(os.path.join("model_training","assets","tag_icons","legacy.png"))
        
    if price == "special": rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_mythic.png"))
    if price == 3250: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_ultimate.png"))
    if price == 1820: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_legendary.png"))
    if price == 1350: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_epic.png"))

    # 1.B Resize rarity and legacy icons to 395x395 pixel resolution overlay images
    if 'legacy_icon' in locals():
        legacy_icon = cv2.resize(legacy_icon, (110,110))
        legacy_icon = cv2.copyMakeBorder(legacy_icon, 291, 0, 0, 297, cv2.BORDER_CONSTANT,value=(0,0,0,0))
        legacy_icon = legacy_icon[0:395,legacy_icon.shape[1]-395:legacy_icon.shape[1]]

    if 'rarity_icon' in locals():
        rarity_icon = cv2.resize(rarity_icon, (385,int(385/rarity_icon.shape[1]*rarity_icon.shape[0])))
        rarity_icon = cv2.copyMakeBorder(rarity_icon, 285, 5, 5, 5, cv2.BORDER_CONSTANT,value=(0,0,0,0))
        rarity_icon = rarity_icon[0:395,0:395]

    # 2. Set up borders this asset uses in the loot tab
    if champion_id.endswith("000"):
        borders = (shard_border, permanent_border, token_6_border, token_7_border)
    else:
        borders = (shard_border, permanent_border)
    
    # 3. Create variations of assets from layers
    for border_index in range(len(borders)):
        for shadow in (False, True):
            image = layer(champion, quantity_background) if shadow else champion
            image = layer(image, borders[border_index])
            image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(14,14,14,255))            

            y_range_increment = 2
            if not champion_id.endswith("000"):
                y_range_increment = 1
                if 'legacy_icon' in locals(): image = layer(image, legacy_icon)  
                if 'rarity_icon' in locals(): image = layer(image, rarity_icon)  
                    
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

        
            # Create different crops to accomodate the imperfection of client screenshot percision
            for x in range(0, 13, 2):
                for y in range(1, 13, y_range_increment):
                    crop = image[0+y:image.shape[1]-10+y, 0+x:image.shape[1]-10+x]              
                    resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                                            
                    destination = os.path.join("model_training","training_data",asset_folder,champion_id,f'{("shard","permanent","token6","token7")[border_index]}_{"yes" if shadow else "no"}-shadow_x{x}_y{y}.png')

                    if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                        os.makedirs(destination.rsplit(os.path.sep,1)[0])

                    cv2.imwrite(destination, resized_image)



# First import libraries.
from PIL import Image
import matplotlib.pyplot as plt

# The folliwing line is useful in Jupyter notebook
%matplotlib inline


def generate_ward_skin_training_images( ward_image_path ):

    ward_id = ward_image_path.rsplit("_",1)[1].split(".")[0]

    if len(glob.glob(os.path.join("model_training","training_data","wards",ward_id,"*.png"))) == 336:
        return

    # 1.A loading and preparing assets
    ward = read_png(ward_image_path)
    ward_shape = np.shape(ward)
    
    # If shape doesn't match, we're crop resizing to fit the rest of the wards
    if ( ward_shape[0] > 550 or ward_shape[1] > 460):
        if ( ward_shape[1]/ward_shape[0] > 460/550):
            # image ratio is wider than targer
            ward = cv2.resize(ward, (int(550/ward_shape[0]*ward_shape[1]) ,550))
            padding = (np.shape(ward)[1]-460)//2
            ward = ward[0:550,padding:460+padding]
        else:
            # image ratio is taller than target (untested - there was no use case for this)
            ward = cv2.resize(ward, (460, int(460/ward_shape[1]*ward_shape[0])))
            padding = (np.shape(ward)[0]-550)//2
            ward = ward[padding:550+padding,0:460]
        plt.imshow(Image.fromarray(np.uint8(ward)))
            
    ward = cv2.copyMakeBorder(ward[0:ward.shape[1],:], 0, 7, 7, 0, cv2.BORDER_CONSTANT,value=(0,0,0,0))
    ward_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","wardskin_background.png")), (467,467))
    shard_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","shard.png")), (467,467))
    permanent_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","permanent.png")), (467,467))
    quantity_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","quantity_background.png")), (467,467))


    borders = (shard_border, permanent_border)
        
    # 2. Create variations of assets from layers
    for border_index in range(len(borders)):
        for shadow in (False, True):
            image = layer(ward_background, quantity_background) if shadow else ward_background
            image = layer(image, ward)
            image = layer(image, borders[border_index])
            image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(14,14,14,255))
                    
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
        
            # Create different crops to accomodate the imperfection of client screenshot percision
            for x in range(0, 13, 2):
                for y in range(1, 13, 1):
                    crop = image[0+y:image.shape[1]-10+y, 0+x:image.shape[1]-10+x]              
                    resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)

                    destination = os.path.join("model_training","training_data","wards",ward_id,f'{("shard","permanent")[border_index]}_{"yes" if shadow else "no"}-shadow_x{x}_y{y}.png')

                    if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                        os.makedirs(destination.rsplit(os.path.sep,1)[0])

                    cv2.imwrite(destination, resized_image)

3. Image generation has begun


In [78]:
# Get champion and skins assets
champions_and_skins = glob.glob(os.path.join("model_training","assets","champions_and_skins","*.jpg"))

# Generate training data if the lookup_loot contains the ID
# If ID is missing in lookup table run "generate_lookup_loot.py"
# If ID is still missing, this champion is very fresh as wikipedia doesn't know of him :)
missing_ids = []

for i in progressbar(range(len(champions_and_skins)), "-  Generating champion/skin images: ", 40):
    image = champions_and_skins[i]
    champion_id = image.rsplit(os.path.sep,1)[1].split('.')[0]
    champion_data = list(filter(lambda row: row[0] == champion_id, lookup_loot["champions"]+lookup_loot["skins"]))
    if len(champion_data) > 0:
        generate_champion_and_skin_training_images(image)
    else:
        missing_ids.append(champion_id)

if len(missing_ids) > 0:
    print("!   Missing asset data. Perhaps a new champion/skin has been released?")
    print("!   The model will not be able to classify this ")
    print("!   IDs with missing data: "+",".join(map(lambda x: str(x), missing_ids)))
    if input("Would you like to continue? (y/n)").lower() == "n":
        exit()

# Get ward skin assets while ignoring their shadow png image
ward_skins = glob.glob(os.path.join("model_training","assets","ward_skins","*.png"))
ward_skins = list(filter(lambda ward: "shadow" not in ward, ward_skins))

# Generate training data if the lookup_loot contains the ID
# If ID is missing in lookup table run action 1 again.
# If ID is still missing, this champion is very fresh as wikipedia doesn't know of him :)
missing_ids = []

for i in progressbar(range(len(ward_skins)), "-  Generating ward skin images: ", 40):
    image = ward_skins[i]
    ward_id = image.rsplit("_",1)[1].split(".")[0]
    ward_data = list(filter(lambda row: row[0] == int(ward_id), lookup_loot["wards"]))
    if len(ward_data) > 0:
        generate_ward_skin_training_images(image)
    else:
        missing_ids.append(ward_id)

if len(missing_ids) > 0:
    print("!   Missing asset data. Perhaps a new ward has been released?")
    print("!   The model will not be able to classify this asset.")
    print("!   IDs with missing data: "+",".join(map(lambda x: str(x), missing_ids)))
    if input("Would you like to continue? (y/n)").lower() == "n":
        exit()

def generate_collection_champion_training_images( champion_image_path ):
        
    champion_id = champion_image_path.rsplit(os.path.sep,1)[1].split('.')[0]
    
    if len(glob.glob(os.path.join("model_training","training_data","collection_champions",champion_id,"*.png"))) == 49:
        return
    
    image = read_png(champion_image_path)[0:377,:]

    bg = np.zeros((image.shape[0], image.shape[1], image.shape[2]), np.uint8)
    bg[:] = (22, 13, 1, 255)

    image = layer(bg, image, 0.8)    
    image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(30,35,40,255))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

    for x in range(0, 13, 2):
        for y in range(0, 13, 2):
            crop = image[0+y:image.shape[0]-10+y, 0+x:image.shape[1]-10+x]              
            resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                    
            destination = os.path.join("model_training","training_data","collection_champions",champion_id,f'x{x}_y{y}.png')

            if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                os.makedirs(destination.rsplit(os.path.sep,1)[0])

            cv2.imwrite(destination, resized_image)

coll_champion = glob.glob(os.path.join("model_training","assets","loading_screen_assets","*.png"))
coll_champion = list(filter(lambda path: path.endswith("000.png"), coll_champion))


for i in progressbar(range(len(coll_champion)), "-  Generating collection champion images: ", 40):
    path = coll_champion[i]
    generate_collection_champion_training_images(path)



def generate_collection_skin_training_images( skin_image_path ):    
    skin_id = skin_image_path.rsplit(os.path.sep,1)[1].split('.')[0]

    if len(glob.glob(os.path.join("model_training","training_data","collection_skins",skin_id,"*.png"))) == 49:
        return
    
    image = read_png(skin_image_path)[18:409,14:291]

    bg = np.zeros((image.shape[0], image.shape[1], image.shape[2]), np.uint8)
    bg[:] = (22, 13, 1, 255)

    image = layer(bg, image, 0.8)    
    image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(30,35,40,255))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

    for x in range(0, 13, 2):
        for y in range(0, 13, 2):
            crop = image[0+y:image.shape[0]-10+y, 0+x:image.shape[1]-10+x]              
            resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                    
            destination = os.path.join("model_training","training_data","collection_skins",skin_id,f'x{x}_y{y}.png')

            if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                os.makedirs(destination.rsplit(os.path.sep,1)[0])

            cv2.imwrite(destination, resized_image)


coll_skin = glob.glob(os.path.join("model_training","assets","loading_screen_assets","*.png"))
coll_skin = list(filter(lambda path: not path.endswith("000.png"), coll_skin))
    
for i in progressbar(range(len(coll_skin)), "-  Generating collection skin images: ", 40):
    path = coll_skin[i]
    generate_collection_skin_training_images(path)


print("-  Image generation complete")

-  Generating champion/skin images: [########################################] 1464/1464
-  Generating ward skin images: [########################################] 220/220
!   Missing asset data. Perhaps a new ward has been released?
!   The model will not be able to classify this asset.
!   IDs with missing data: 221
-  Generating collection champion images: [########################################] 157/157
-  Generating collection skin images: [########################################] 1651/1651
-  Image generation complete
